In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az
import matplotlib.pyplot as plt
import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

In [2]:
import pandas as pd
from IPython.display import display
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [4]:


coord_to_columns = load.CoordToColumns(
    time='time',
    geo='geo',
    controls=['GQV', 'Competitor_Sales'],
    population='population',
    kpi='conversions',
    revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
        'Channel3_impression',
        'Channel4_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
        'Channel3_spend',
        'Channel4_spend',
    ],
    organic_media=['Organic_channel0_impression'],
    non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'Channel0_impression': 'Channel_0',
    'Channel1_impression': 'Channel_1',
    'Channel2_impression': 'Channel_2',
    'Channel3_impression': 'Channel_3',
    'Channel4_impression': 'Channel_4',
}
correct_media_spend_to_channel = {
    'Channel0_spend': 'Channel_0',
    'Channel1_spend': 'Channel_1',
    'Channel2_spend': 'Channel_2',
    'Channel3_spend': 'Channel_3',
    'Channel4_spend': 'Channel_4',
}


loader = load.CsvDataLoader(
    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000, seed=1)

ValueError: DataFrame is missing one or more columns from ['GQV', 'Competitor_Sales', 'time']

# Steps
1. Load the csv from the test example
2. Translate the data I have into the same format
3. Run the model and see what we get with the parameters we have

In [ ]:
import pandas as pd

In [ ]:
!pwd

/home/ubuntu/work/marketing/google


In [ ]:
example_data = pd.read_csv('./example.csv')

In [ ]:
example_data.head()

,Unnamed: 0,geo,time,Channel0_impression,Channel1_impression,Channel2_impression,Channel3_impression,Channel4_impression,Competitor_Sales,GQV,Channel0_spend,Channel1_spend,Channel2_spend,Channel3_spend,Channel4_spend,Organic_channel0_impression,Promo,conversions,revenue_per_conversion,population
0,0,Geo0,2021-01-25,280668.0,0.0,0.0,470611.0,108010.0,-1.338765,0.115581,2058.0608,0.00000,0.00000,3667.3965,841.6044,97320.0,0.000000,1954576.8,0.020055,136670.94
1,1,Geo0,2021-02-01,366206.0,182108.0,19825.0,527702.0,252506.0,0.893645,0.944224,2685.2874,1755.74540,147.31808,4112.2974,1967.5044,201441.0,0.000000,2064249.6,0.020103,136670.94
2,2,Geo0,2021-02-08,197565.0,230170.0,0.0,393618.0,184061.0,-0.284549,-1.290579,1448.6895,2219.12230,0.00000,3067.4023,1434.1870,0.0,0.683819,2086382.8,0.019929,136670.94
3,3,Geo0,2021-02-15,140990.0,66643.0,0.0,326034.0,201729.0,-1.034740,-1.084514,1033.8406,642.52057,0.00000,2540.7310,1571.8545,0.0,1.289055,2826431.5,0.019987,136670.94
4,4,Geo0,2021-02-22,399116.0,164991.0,0.0,381982.0,153973.0,-0.319276,-0.017503,2926.6072,1590.71640,0.00000,2976.7249,1199.7440,0.0,0.227739,3551929.2,0.020000,136670.94


In [ ]:
example_data['time'].value_counts()

time
2024-01-15    40
2021-01-25    40
2021-02-01    40
2021-02-08    40
2023-10-02    40
              ..
2021-03-15    40
2021-03-22    40
2021-03-29    40
2021-04-05    40
2021-04-12    40
Name: count, Length: 156, dtype: int64

In [ ]:
example_data[example_data['time']=='2024-01-15']

,Unnamed: 0,geo,time,Channel0_impression,Channel1_impression,Channel2_impression,Channel3_impression,Channel4_impression,Competitor_Sales,GQV,Channel0_spend,Channel1_spend,Channel2_spend,Channel3_spend,Channel4_spend,Organic_channel0_impression,Promo,conversions,revenue_per_conversion,population
155,155,Geo0,2024-01-15,0.0,0.0,0.0,153290.0,0.0,-1.714900,-1.567028,0.00000,0.00000,0.00000,1194.5645,0.000000,0.0,0.000000,2261657.5,0.019996,136670.94
311,311,Geo1,2024-01-15,0.0,165884.0,0.0,878546.0,281031.0,1.442481,-0.582913,0.00000,1599.32600,0.00000,6846.3690,2189.768800,95607.0,0.000000,589451.4,0.020141,199816.90
467,467,Geo10,2024-01-15,1223700.0,0.0,0.0,5752647.0,1424580.0,-2.088481,1.279819,8973.05300,0.00000,0.00000,44829.4600,11100.201000,721022.0,0.000000,26094610.0,0.020114,994048.94
623,623,Geo11,2024-01-15,2051320.0,0.0,1817748.0,2815561.0,634036.0,-0.220375,1.692105,15041.76200,0.00000,13507.54800,21941.2190,4940.352500,1762694.0,1.096069,11311549.0,0.020000,659730.00
779,779,Geo12,2024-01-15,0.0,0.0,0.0,2606966.0,1334920.0,-0.296051,-1.539180,0.00000,0.00000,0.00000,20315.6700,10401.579000,0.0,0.000000,8941670.0,0.020077,560989.75
935,935,Geo13,2024-01-15,296490.0,361494.0,0.0,2158874.0,0.0,-0.988700,0.079305,2174.07900,3485.24730,0.00000,16823.7600,0.000000,0.0,0.537346,12305039.0,0.019815,444255.62
1091,1091,Geo14,2024-01-15,802547.0,402498.0,473561.0,3821243.0,546744.0,0.792236,0.837544,5884.85500,3880.57640,3518.99600,29778.3380,4260.181000,1016860.0,0.000000,11464448.0,0.019843,765448.10
1247,1247,Geo15,2024-01-15,2038318.0,0.0,236781.0,834364.0,778196.0,0.337785,-0.145998,14946.42200,0.00000,1759.50170,6502.0654,6063.634300,294836.0,0.078605,18190758.0,0.020001,680241.75
1403,1403,Geo16,2024-01-15,1056231.0,637242.0,758455.0,2060982.0,543317.0,0.368793,3.521468,7745.04930,6143.79740,5636.02200,16060.9040,4233.478000,159814.0,0.000000,4956941.0,0.020083,376367.94
1559,1559,Geo17,2024-01-15,207126.0,310816.0,449943.0,3486966.0,2001128.0,-0.301321,1.211981,1518.79760,2996.64900,3343.49240,27173.3700,15592.612000,1281502.0,0.000000,12121405.0,0.020032,777539.94


In [ ]:
# What is the strucrure of the example that I have? Go and have a look!
example_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6240 entries, 0 to 6239
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   6240 non-null   int64  
 1   geo                          6240 non-null   object 
 2   time                         6240 non-null   object 
 3   Channel0_impression          6240 non-null   float64
 4   Channel1_impression          6240 non-null   float64
 5   Channel2_impression          6240 non-null   float64
 6   Channel3_impression          6240 non-null   float64
 7   Channel4_impression          6240 non-null   float64
 8   Competitor_Sales             6240 non-null   float64
 9   GQV                          6240 non-null   float64
 10  Channel0_spend               6240 non-null   float64
 11  Channel1_spend               6240 non-null   float64
 12  Channel2_spend               6240 non-null   float64
 13  Channel3_spend    

In [6]:
!pwd

/Users/aaronmeagher/Work/google_meridian/google/ws_attempt


In [3]:
# Import of our data

from data.data_aquisition import *

In [4]:
ws_data = local_import_data()

/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/data_aquisition.py:22: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend = pd.read_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/advertising_raw.csv')


In [5]:
revenue, impressions = ws_data

In [6]:
impressions.head()

,google_campaign_spend_eur,facebook_campaign_spend_eur,twitter_campaign_spend_eur,ticktock_campaign_spend_eur,instagram_campaign_spend_eur,reddit_campaign_spend_eur,date_week
2022-01-02,111.034778,0.0,0.0,0.0,0.0,0.0,2022-01-02
2022-01-03,285.447340,0.0,0.0,0.0,0.0,0.0,2022-01-03
2022-01-04,522.218717,0.0,0.0,0.0,0.0,0.0,2022-01-04
2022-01-05,471.739257,0.0,0.0,0.0,0.0,0.0,2022-01-05
2022-01-06,556.178809,0.0,0.0,0.0,0.0,0.0,2022-01-06


In [7]:
raw_marketing_data = pd.read_csv('./ws_attempt/data/raw_data/advertising_raw.csv')

FileNotFoundError: [Errno 2] No such file or directory: './ws_attempt/data/raw_data/advertising_raw.csv'

In [ ]:
raw_marketing_data.head()

,Unnamed: 0,marketing_campaign_name,event_id,event_salesforce_conference_ids,conference_year,event_currency_id,currency_id,event_date,impressions,parent_conference,...,marketing_platform_name,event_avenger_slug,last_ingestion_timestamp,weeks_to_event,event_name,classification_group,marketing_campaign_id,event_tito_slug,reach_country_id,marketing_ad_account_name
0,0,Conf=Collision2023&Activity=open_source&Object...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,24817,Collision,...,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,General Attendee- Subsid,1684929535866970982,unknown,---,CollisionConf
1,1,Conf=Collision2023&Activity=alpha&Objective=Le...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,6191,Collision,...,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,Startups,1689310747829239719,unknown,---,CollisionConf
2,2,Conf=WebSummit2022&Activity=general_attendee&O...,f0009977-13e9-460a-9d34-3efca62a3b36,[a0J7R00000e9KOVUA2],2022,1,2,2022-11-01,613019,Web Summit,...,Reddit,ws22,2024-10-07T03:50:02.858,7.0,Web Summit 2022,General Attendee,1523191475961491496,unknown,---,WebSummitConf
3,3,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,39309,Collision,...,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf
4,4,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,102750,Collision,...,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf


In [ ]:
marketing_spend_raw = pd.read_csv('/home/ubuntu/work/marketing/marketing_analytics/functionalised/data/raw_data/advertising_raw.csv')

/tmp/ipykernel_201732/2953436655.py:1: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend_raw = pd.read_csv('/home/ubuntu/work/marketing/marketing_analytics/functionalised/data/raw_data/advertising_raw.csv')


In [ ]:
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    min_date = min(marketing_spend['date_id'])
    max_date = max(marketing_spend['date_id'])
    overall_date_range = pd.date_range(min_date, max_date)
    overall_date_range
    marketing_spend.index
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    facebook = marketing_spend[marketing_spend['marketing_platform_name']=='FacebookAds'].add_prefix('facebook_')
    linkedin = marketing_spend[marketing_spend['marketing_platform_name']=='LinkedInAds'].add_prefix('linkedin_')
    google = marketing_spend[marketing_spend['marketing_platform_name']=='GoogleAds'].add_prefix('google_')
    bing = marketing_spend[marketing_spend['marketing_platform_name']=='BingAds'].add_prefix('bing_')
    ticktock = marketing_spend[marketing_spend['marketing_platform_name']=='TikTok'].add_prefix('ticktock_')
    twitter = marketing_spend[marketing_spend['marketing_platform_name']=='TwitterAds'].add_prefix('twitter_')    
    instagram = marketing_spend[marketing_spend['marketing_platform_name']=='Instagram'].add_prefix('instagram_')
    reddit = marketing_spend[marketing_spend['marketing_platform_name']=='Reddit'].add_prefix('reddit_')


    #facebook['facebook_date_id'] = pd.to_datetime(facebook['facebook_date_id'])
    facebook = facebook.set_index(facebook['facebook_date_id'])
    facebook = facebook[['facebook_impressions','facebook_reach_country_id','facebook_campaign_spend_eur']]
    facebook_clean = facebook.groupby(facebook.index).sum()

    linkedin = linkedin.set_index(linkedin['linkedin_date_id'])
    linkedin = linkedin[['linkedin_impressions','linkedin_reach_country_id','linkedin_campaign_spend_eur']]
    linkedin_clean = linkedin.groupby(linkedin.index).sum()

    google = google.set_index(google['google_date_id'])
    google = google[['google_impressions','google_reach_country_id','google_campaign_spend_eur']]
    google_clean = google.groupby(google.index).sum()

    bing = bing.set_index(bing['bing_date_id'])
    bing = bing[['bing_impressions','bing_reach_country_id','bing_campaign_spend_eur']]
    bing_clean = bing.groupby(bing.index).sum()

    ticktock = ticktock.set_index(ticktock['ticktock_date_id'])
    ticktock = ticktock[['ticktock_impressions','ticktock_reach_country_id','ticktock_campaign_spend_eur']]
    ticktock_clean = ticktock.groupby(ticktock.index).sum()

    twitter = twitter.set_index(twitter['twitter_date_id'])
    twitter = twitter[['twitter_impressions','twitter_reach_country_id','twitter_campaign_spend_eur']]
    twitter_clean = twitter.groupby(twitter.index).sum()

    instagram = instagram.set_index(instagram['instagram_date_id'])
    instagram = instagram[['instagram_impressions','instagram_reach_country_id','instagram_campaign_spend_eur']]
    instagram_clean = instagram.groupby(instagram.index).sum()

    reddit = reddit.set_index(reddit['reddit_date_id'])
    reddit = reddit[['reddit_impressions','reddit_reach_country_id','reddit_campaign_spend_eur']]
    reddit_clean = reddit.groupby(reddit.index).sum()

    combined = pd.concat([facebook_clean, linkedin_clean, google_clean, bing_clean,
                     ticktock_clean, twitter_clean, instagram_clean, reddit_clean],
                    axis=1)



    #return [google, facebook, twitter, ticktock, instagram, reddit, bing, linkedin, ticket_sales]

SyntaxError: 'return' outside function (1158002199.py, line 56)

In [ ]:
from functools import reduce
suffixes = ['_fb', '_li', '_go', '_bi', '_tt', '_tw', '_ig', '_rd']
dfs = [facebook_clean, linkedin_clean, google_clean, bing_clean,
       ticktock_clean, twitter_clean, instagram_clean, reddit_clean]
merged_df = reduce(lambda left, right: pd.merge(
    left, 
    right, 
    on='date', 
    how='outer',
    suffixes=('', suffixes[dfs.index(right)])
), dfs)

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [ ]:
print("Google duplicates:", google.index.duplicated().any())


Google duplicates: True


In [ ]:
combined.columns

Index(['facebook_impressions', 'facebook_reach_country_id',
       'facebook_campaign_spend_eur', 'linkedin_impressions',
       'linkedin_reach_country_id', 'linkedin_campaign_spend_eur',
       'google_impressions', 'google_reach_country_id',
       'google_campaign_spend_eur', 'bing_impressions',
       'bing_reach_country_id', 'bing_campaign_spend_eur',
       'ticktock_impressions', 'ticktock_reach_country_id',
       'ticktock_campaign_spend_eur', 'twitter_impressions',
       'twitter_reach_country_id', 'twitter_campaign_spend_eur',
       'instagram_impressions', 'instagram_reach_country_id',
       'instagram_campaign_spend_eur', 'reddit_impressions',
       'reddit_reach_country_id', 'reddit_campaign_spend_eur'],
      dtype='object')

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1010 entries, 2022-01-02 to 2024-10-07
Freq: D
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   facebook_impressions          942 non-null    float64
 1   facebook_reach_country_id     942 non-null    object 
 2   facebook_campaign_spend_eur   942 non-null    float64
 3   linkedin_impressions          996 non-null    float64
 4   linkedin_reach_country_id     996 non-null    object 
 5   linkedin_campaign_spend_eur   996 non-null    float64
 6   google_impressions            1010 non-null   int64  
 7   google_reach_country_id       1010 non-null   object 
 8   google_campaign_spend_eur     1010 non-null   float64
 9   bing_impressions              967 non-null    float64
 10  bing_reach_country_id         967 non-null    object 
 11  bing_campaign_spend_eur       967 non-null    float64
 12  ticktock_impressions          951 no

In [ ]:
combined.describe()

,facebook_impressions,facebook_campaign_spend_eur,linkedin_impressions,linkedin_campaign_spend_eur,google_impressions,google_campaign_spend_eur,bing_impressions,bing_campaign_spend_eur,ticktock_impressions,ticktock_campaign_spend_eur,twitter_impressions,twitter_campaign_spend_eur,instagram_impressions,instagram_campaign_spend_eur,reddit_impressions,reddit_campaign_spend_eur
count,9.420000e+02,942.000000,9.960000e+02,996.000000,1.010000e+03,1010.000000,967.000000,967.000000,9.510000e+02,951.000000,2.440000e+02,244.000000,12.000000,12.000000,83.000000,83.000000
mean,2.445419e+06,5819.252134,5.448231e+05,11066.633296,8.107539e+05,3895.801130,55540.781799,384.139369,2.788839e+05,455.055664,3.680436e+05,386.909552,96345.416667,94.251667,148724.638554,271.834996
std,2.858536e+06,4487.308994,4.334113e+05,8300.061435,1.212501e+06,3025.696803,100094.941430,330.947481,5.464332e+05,852.102840,1.091628e+06,674.711584,181129.481208,89.147147,97448.738480,225.013915
min,2.000000e+00,0.000000,0.000000e+00,0.000000,6.930000e+02,30.019889,0.000000,0.000000,0.000000e+00,0.000000,1.000000e+00,0.000000,1563.000000,6.940000,2.000000,0.000000
25%,4.541855e+05,2364.164463,2.183298e+05,4670.521306,9.612100e+04,1767.092263,3665.000000,82.235000,0.000000e+00,0.000000,4.411850e+04,89.812284,12591.000000,24.680000,89268.000000,196.004561
50%,1.392734e+06,4919.312676,4.586395e+05,9964.080136,4.058730e+05,3214.985387,12662.000000,300.640000,1.975300e+04,90.909951,1.085150e+05,250.000000,22838.500000,51.410000,146500.000000,230.617577
75%,3.497515e+06,8345.114534,7.563912e+05,14982.567070,9.304572e+05,5258.868858,40547.000000,575.330000,3.476265e+05,537.928148,2.751372e+05,420.459460,36637.000000,147.430000,181605.500000,291.853737
max,1.798800e+07,29942.039996,2.818737e+06,46124.302431,9.382256e+06,22751.928547,741305.000000,1663.740000,4.351418e+06,9682.273009,8.045494e+06,6663.855450,544716.000000,245.720000,613019.000000,1828.317769


In [ ]:
combined.head()

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,linkedin_impressions,linkedin_reach_country_id,linkedin_campaign_spend_eur,google_impressions,google_reach_country_id,google_campaign_spend_eur,bing_impressions,...,ticktock_campaign_spend_eur,twitter_impressions,twitter_reach_country_id,twitter_campaign_spend_eur,instagram_impressions,instagram_reach_country_id,instagram_campaign_spend_eur,reddit_impressions,reddit_reach_country_id,reddit_campaign_spend_eur
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,693,PRTMOZKORZAFEGYZAFNORUSADNKSRBBGDROUBELNLDLKAF...,111.034778,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,NaN,NaN,NaN,0.0,---,0.0,1398,ARGLBNESTFRASRBMKDSVNURYGBRPERNPLISRRUSGRCPRIS...,285.447340,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,0.0,---,0.0,2478,KENLTUGEOBELMUSVNMUKRNGAIRLNGAIDNHNDPRTRUSFRAC...,522.218717,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,0.0,---,0.0,2317,INDJPNUKRCANFRAMEXTZALKAVENBELEGYHRVARGAGOPAKN...,471.739257,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,3120,NLDGHAFRAUSAIRLSAUGBRCANLTUPHLPOLNORFINTHACOLS...,556.178809,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
reddit = marketing_spend[marketing_spend['marketing_platform_name']=='Reddit'].add_prefix('reddit_')

In [ ]:
reddit.columns

Index(['reddit_Unnamed: 0', 'reddit_marketing_campaign_name',
       'reddit_event_id', 'reddit_event_salesforce_conference_ids',
       'reddit_conference_year', 'reddit_event_currency_id',
       'reddit_currency_id', 'reddit_event_date', 'reddit_impressions',
       'reddit_parent_conference', 'reddit_previous_year_event_id',
       'reddit_next_year_event_id', 'reddit_classification_sub_group',
       'reddit_event_tito_id', 'reddit_classification_id',
       'reddit_marketing_platform_id', 'reddit_clicks', 'reddit_date_id',
       'reddit_reach', 'reddit_event_salesforce_campaign_ids',
       'reddit_event_avenger_id', 'reddit_campaign_spend_eur',
       'reddit_days_to_event', 'reddit_marketing_campaign_objective',
       'reddit_campaign_spend', 'reddit_marketing_ad_account_id',
       'reddit_marketing_platform_name', 'reddit_event_avenger_slug',
       'reddit_last_ingestion_timestamp', 'reddit_weeks_to_event',
       'reddit_event_name', 'reddit_classification_group',
       

# Find below the update to the data processing section to get it to resemble the data from the google-bmmm model

In [10]:
ws_data = local_import_data()

/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/data_aquisition.py:22: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend = pd.read_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/advertising_raw.csv')


In [ ]:
marketing_spend= pd.read_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/advertising_raw.csv')

/tmp/ipykernel_201732/1607298651.py:1: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend= pd.read_csv('/home/ubuntu/work/marketing/marketing_analytics/functionalised/data/raw_data/advertising_raw.csv')


In [9]:
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    min_date = min(marketing_spend['date_id'])
    max_date = max(marketing_spend['date_id'])
    overall_date_range = pd.date_range(min_date, max_date)
    overall_date_range
    marketing_spend.index
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    facebook = marketing_spend[marketing_spend['marketing_platform_name']=='FacebookAds'].add_prefix('facebook_')
    linkedin = marketing_spend[marketing_spend['marketing_platform_name']=='LinkedInAds'].add_prefix('linkedin_')
    google = marketing_spend[marketing_spend['marketing_platform_name']=='GoogleAds'].add_prefix('google_')
    bing = marketing_spend[marketing_spend['marketing_platform_name']=='BingAds'].add_prefix('bing_')
    ticktock = marketing_spend[marketing_spend['marketing_platform_name']=='TikTok'].add_prefix('ticktock_')
    twitter = marketing_spend[marketing_spend['marketing_platform_name']=='TwitterAds'].add_prefix('twitter_')    
    instagram = marketing_spend[marketing_spend['marketing_platform_name']=='Instagram'].add_prefix('instagram_')
    reddit = marketing_spend[marketing_spend['marketing_platform_name']=='Reddit'].add_prefix('reddit_')


    #facebook['facebook_date_id'] = pd.to_datetime(facebook['facebook_date_id'])
    #facebook = facebook.set_index(facebook['facebook_date_id'])
    facebook = facebook[['facebook_impressions','facebook_reach_country_id','facebook_campaign_spend_eur', 'facebook_date_id']]
    #facebook_clean = facebook.groupby(facebook.index).sum()

    #linkedin = linkedin.set_index(linkedin['linkedin_date_id'])
    linkedin = linkedin[['linkedin_impressions','linkedin_reach_country_id','linkedin_campaign_spend_eur', 'linkedin_date_id']]
    #linkedin_clean = linkedin.groupby(linkedin.index).sum()

    #google = google.set_index(google['google_date_id'])
    google = google[['google_impressions','google_reach_country_id','google_campaign_spend_eur', 'google_date_id']]
    #google_clean = google.groupby(google.index).sum()

    #bing = bing.set_index(bing['bing_date_id'])
    bing = bing[['bing_impressions','bing_reach_country_id','bing_campaign_spend_eur', 'bing_date_id']]
    #bing_clean = bing.groupby(bing.index).sum()

    #ticktock = ticktock.set_index(ticktock['ticktock_date_id'])
    ticktock = ticktock[['ticktock_impressions','ticktock_reach_country_id','ticktock_campaign_spend_eur', 'ticktock_date_id']]
    #ticktock_clean = ticktock.groupby(ticktock.index).sum()

    #twitter = twitter.set_index(twitter['twitter_date_id'])
    twitter = twitter[['twitter_impressions','twitter_reach_country_id','twitter_campaign_spend_eur', 'twitter_date_id']]
    #twitter_clean = twitter.groupby(twitter.index).sum()

    #instagram = instagram.set_index(instagram['instagram_date_id'])
    instagram = instagram[['instagram_impressions','instagram_reach_country_id','instagram_campaign_spend_eur','instagram_date_id']]
    #instagram_clean = instagram.groupby(instagram.index).sum()

    #reddit = reddit.set_index(reddit['reddit_date_id'])
    reddit = reddit[['reddit_impressions','reddit_reach_country_id','reddit_campaign_spend_eur', 'reddit_date_id']]
    #reddit_clean = reddit.groupby(reddit.index).sum()

    #combined_google_2 = pd.concat([facebook_clean, linkedin_clean, google_clean, bing_clean,
    #                 ticktock_clean, twitter_clean, instagram_clean, reddit_clean],
    #                axis=1)

    combined_google = pd.concat([facebook, linkedin, google, bing,
                     ticktock, twitter, instagram, reddit],
                    axis=1)

    #return [google, facebook, twitter, ticktock, instagram, reddit, bing, linkedin, ticket_sales]

NameError: name 'marketing_spend' is not defined

In [ ]:
reddit.columns

Index(['reddit_impressions', 'reddit_reach_country_id',
       'reddit_campaign_spend_eur', 'reddit_date_id'],
      dtype='object')

In [ ]:
facebook.head()

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,facebook_date_id
181,12,MLI,0.00,2022-09-16
182,26,TZA,0.03,2022-05-22
183,21,BEL,0.60,2022-04-11
184,14,PER,0.10,2022-02-22
185,11,AUS,0.35,2022-03-27


In [ ]:
combined_google.head()

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,facebook_date_id,linkedin_impressions,linkedin_reach_country_id,linkedin_campaign_spend_eur,linkedin_date_id,google_impressions,google_reach_country_id,google_campaign_spend_eur,google_date_id,bing_impressions,bing_reach_country_id,bing_campaign_spend_eur,bing_date_id,ticktock_impressions,ticktock_reach_country_id,ticktock_campaign_spend_eur,ticktock_date_id,twitter_impressions,twitter_reach_country_id,twitter_campaign_spend_eur,twitter_date_id,instagram_impressions,instagram_reach_country_id,instagram_campaign_spend_eur,instagram_date_id,reddit_impressions,reddit_reach_country_id,reddit_campaign_spend_eur,reddit_date_id
181,12.0,MLI,0.00,2022-09-16,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
182,26.0,TZA,0.03,2022-05-22,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
183,21.0,BEL,0.60,2022-04-11,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
184,14.0,PER,0.10,2022-02-22,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT
185,11.0,AUS,0.35,2022-03-27,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT


In [ ]:


coord_to_columns = load.CoordToColumns(
    time='date_id',
    geo='facebook_reach_country_id',
    controls=[],
    #population='population',
    kpi='clicks',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'linkedin_impressions',
        'google_impressions',
        'bing_impressions',
        'ticktock_impressions',
        'twitter_impressions'
        'reddit_impressions',
        'instagram_impressions',
    ],
    media_spend=[
        'facebook_campaign_spend_eur',
        'linkedin_campaign_spend_eur',
        'google_campaign_spend_eur',
        'bing_campaign_spend_eur',
        'ticktock_campaign_spend_eur',
        'twitter_campaign_spend_eur',
        'reddit_campaign_spend_eur',
        'instagram_campaign_spend_eur',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'linkedin_impressions': 'LinkedIn',
    'google_impressions': 'Google',
    'bing_impressions': 'Bing',
    'ticktock_impressions': 'TickTock',
    'twitter_impressions': 'Twitter',
    'reddit_impressions': 'Reddit',
    'instagram_impressions': 'Instagram',
}
correct_media_spend_to_channel = {
    'facebook_campaign_spend_eur': 'Facebook',
    'linkedin_campaign_spend_eur': 'LinkedIn',
    'google_campaign_spend_eur': 'Google',
    'bing_campaign_spend_eur': 'Bing',
    'ticktock_campaign_spend_eur': 'TickTock',
    'twitter_campaign_spend_eur': 'Twitter',
    'reddit_campaign_spend_eur': 'Reddit',
    'instagram_campaign_spend_eur': 'Instagram',
}


#loader = load.CsvDataLoader(
#    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
#    kpi_type='non_revenue',
#    coord_to_columns=coord_to_columns,
#    media_to_channel=correct_media_to_channel,
#    media_spend_to_channel=correct_media_spend_to_channel,
#)

loader = load.CsvDataLoader(
    csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/advertising_raw.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


data = loader.load()

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000, seed=1)

/home/ubuntu/.local/lib/python3.10/site-packages/meridian/data/load.py:1384: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


ValueError: Values of the `coord_to_columns` object ['bing_campaign_spend_eur', 'bing_impressions', 'clicks', 'date_id', 'facebook_campaign_spend_eur', 'facebook_impressions', 'facebook_reach_country_id', 'google_campaign_spend_eur', 'google_impressions', 'instagram_campaign_spend_eur', 'instagram_impressions', 'linkedin_campaign_spend_eur', 'linkedin_impressions', 'population', 'reddit_campaign_spend_eur', 'ticktock_campaign_spend_eur', 'ticktock_impressions', 'twitter_campaign_spend_eur', 'twitter_impressionsreddit_impressions'] should map to the DataFrame column names ['Unnamed: 0', 'campaign_spend', 'campaign_spend_eur', 'classification_group', 'classification_id', 'classification_sub_group', 'clicks', 'conference_year', 'currency_id', 'date_id', 'days_to_event', 'event_avenger_id', 'event_avenger_slug', 'event_currency_id', 'event_date', 'event_id', 'event_name', 'event_salesforce_campaign_ids', 'event_salesforce_conference_ids', 'event_tito_id', 'event_tito_slug', 'facebook_reach

In [ ]:


coord_to_columns = load.CoordToColumns(
    time='date_id',
    geo='facebook_reach_country_id',
    controls=[],
    #population='population',
    kpi='clicks',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'linkedin_impressions',
        'google_impressions',
        'bing_impressions',
        'ticktock_impressions',
        'twitter_impressions'
        'reddit_impressions',
        'instagram_impressions',
    ],
    media_spend=[
        'facebook_campaign_spend_eur',
        'linkedin_campaign_spend_eur',
        'google_campaign_spend_eur',
        'bing_campaign_spend_eur',
        'ticktock_campaign_spend_eur',
        'twitter_campaign_spend_eur',
        'reddit_campaign_spend_eur',
        'instagram_campaign_spend_eur',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'linkedin_impressions': 'LinkedIn',
    'google_impressions': 'Google',
    'bing_impressions': 'Bing',
    'ticktock_impressions': 'TickTock',
    'twitter_impressions': 'Twitter',
    'reddit_impressions': 'Reddit',
    'instagram_impressions': 'Instagram',
}
correct_media_spend_to_channel = {
    'facebook_campaign_spend_eur': 'Facebook',
    'linkedin_campaign_spend_eur': 'LinkedIn',
    'google_campaign_spend_eur': 'Google',
    'bing_campaign_spend_eur': 'Bing',
    'ticktock_campaign_spend_eur': 'TickTock',
    'twitter_campaign_spend_eur': 'Twitter',
    'reddit_campaign_spend_eur': 'Reddit',
    'instagram_campaign_spend_eur': 'Instagram',
}


#loader = load.CsvDataLoader(
#    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
#    kpi_type='non_revenue',
#    coord_to_columns=coord_to_columns,
#    media_to_channel=correct_media_to_channel,
#    media_spend_to_channel=correct_media_spend_to_channel,
#)

loader = load.CsvDataLoader(
    csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/advertising_raw.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


data = loader.load()

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000, seed=1)

/home/ubuntu/.local/lib/python3.10/site-packages/meridian/data/load.py:1384: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


ValueError: Values of the `coord_to_columns` object ['bing_campaign_spend_eur', 'bing_impressions', 'clicks', 'date_id', 'facebook_campaign_spend_eur', 'facebook_impressions', 'facebook_reach_country_id', 'google_campaign_spend_eur', 'google_impressions', 'instagram_campaign_spend_eur', 'instagram_impressions', 'linkedin_campaign_spend_eur', 'linkedin_impressions', 'population', 'reddit_campaign_spend_eur', 'ticktock_campaign_spend_eur', 'ticktock_impressions', 'twitter_campaign_spend_eur', 'twitter_impressionsreddit_impressions'] should map to the DataFrame column names ['Unnamed: 0', 'campaign_spend', 'campaign_spend_eur', 'classification_group', 'classification_id', 'classification_sub_group', 'clicks', 'conference_year', 'currency_id', 'date_id', 'days_to_event', 'event_avenger_id', 'event_avenger_slug', 'event_currency_id', 'event_date', 'event_id', 'event_name', 'event_salesforce_campaign_ids', 'event_salesforce_conference_ids', 'event_tito_id', 'event_tito_slug', 'facebook_reach

In [ ]:
combined.iloc[0]

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,linkedin_impressions,linkedin_reach_country_id,linkedin_campaign_spend_eur,google_impressions,google_reach_country_id,google_campaign_spend_eur,bing_impressions,...,ticktock_campaign_spend_eur,twitter_impressions,twitter_reach_country_id,twitter_campaign_spend_eur,instagram_impressions,instagram_reach_country_id,instagram_campaign_spend_eur,reddit_impressions,reddit_reach_country_id,reddit_campaign_spend_eur
181,12.0,MLI,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,26.0,TZA,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,21.0,BEL,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,14.0,PER,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,11.0,AUS,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10008.0,---,42.610874
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9308.0,---,40.579586
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9909.0,---,37.059004
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4550.0,---,13.675289


In [ ]:
combined.iloc[50]

facebook_impressions            514.0
facebook_reach_country_id         VEN
facebook_campaign_spend_eur      0.21
linkedin_impressions              NaN
linkedin_reach_country_id         NaN
linkedin_campaign_spend_eur       NaN
google_impressions                NaN
google_reach_country_id           NaN
google_campaign_spend_eur         NaN
bing_impressions                  NaN
bing_reach_country_id             NaN
bing_campaign_spend_eur           NaN
ticktock_impressions              NaN
ticktock_reach_country_id         NaN
ticktock_campaign_spend_eur       NaN
twitter_impressions               NaN
twitter_reach_country_id          NaN
twitter_campaign_spend_eur        NaN
instagram_impressions             NaN
instagram_reach_country_id        NaN
instagram_campaign_spend_eur      NaN
reddit_impressions                NaN
reddit_reach_country_id           NaN
reddit_campaign_spend_eur         NaN
Name: 231, dtype: object

In [ ]:
marketing_spend_raw.columns

Index(['Unnamed: 0', 'marketing_campaign_name', 'event_id',
       'event_salesforce_conference_ids', 'conference_year',
       'event_currency_id', 'currency_id', 'event_date', 'impressions',
       'parent_conference', 'previous_year_event_id', 'next_year_event_id',
       'classification_sub_group', 'event_tito_id', 'classification_id',
       'marketing_platform_id', 'clicks', 'date_id', 'reach',
       'event_salesforce_campaign_ids', 'event_avenger_id',
       'campaign_spend_eur', 'days_to_event', 'marketing_campaign_objective',
       'campaign_spend', 'marketing_ad_account_id', 'marketing_platform_name',
       'event_avenger_slug', 'last_ingestion_timestamp', 'weeks_to_event',
       'event_name', 'classification_group', 'marketing_campaign_id',
       'event_tito_slug', 'reach_country_id', 'marketing_ad_account_name'],
      dtype='object')

In [ ]:
marketing_spend_raw.head()

,Unnamed: 0,marketing_campaign_name,event_id,event_salesforce_conference_ids,conference_year,event_currency_id,currency_id,event_date,impressions,parent_conference,previous_year_event_id,next_year_event_id,classification_sub_group,event_tito_id,classification_id,marketing_platform_id,clicks,date_id,reach,event_salesforce_campaign_ids,event_avenger_id,campaign_spend_eur,days_to_event,marketing_campaign_objective,campaign_spend,marketing_ad_account_id,marketing_platform_name,event_avenger_slug,last_ingestion_timestamp,weeks_to_event,event_name,classification_group,marketing_campaign_id,event_tito_slug,reach_country_id,marketing_ad_account_name
0,0,Conf=Collision2023&Activity=open_source&Object...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,24817,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,Open Source,unknown,generalattendee-subsid_opensource,8,70.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,147.373190,54.0,CLICKS,161.565228,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,General Attendee- Subsid,1684929535866970982,unknown,---,CollisionConf
1,1,Conf=Collision2023&Activity=alpha&Objective=Le...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,6191,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,ALPHA,unknown,startups_alpha,8,28.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,55.369416,54.0,CLICKS,60.701491,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,Startups,1689310747829239719,unknown,---,CollisionConf
2,2,Conf=WebSummit2022&Activity=general_attendee&O...,f0009977-13e9-460a-9d34-3efca62a3b36,[a0J7R00000e9KOVUA2],2022,1,2,2022-11-01,613019,Web Summit,aebb4e42-a61d-4b12-8850-aa1d5bafb053,494e0d47-2838-43c4-bf1a-12aea8d65194,General Attendee,unknown,generalattendee_generalattendee,8,1516.0,2022-09-10,0,[7012X000001TH0oQAG],f0009977-13e9-460a-9d34-3efca62a3b36,1828.317769,52.0,CLICKS,1852.268732,t2_48ksux2y,Reddit,ws22,2024-10-07T03:50:02.858,7.0,Web Summit 2022,General Attendee,1523191475961491496,unknown,---,WebSummitConf
3,3,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,39309,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,127.0,2022-04-29,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,86.894148,52.0,CLICKS,95.809488,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf
4,4,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,102750,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,331.0,2022-04-28,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,238.412503,53.0,CLICKS,262.873626,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf


In [ ]:
print(marketing_spend_raw.iloc[0])

Unnamed: 0                                                                         0
marketing_campaign_name            Conf=Collision2023&Activity=open_source&Object...
event_id                                        20ed6ee7-0a8f-41b7-8497-a5733dc61431
event_salesforce_conference_ids                                 [a0J7R00000e9R6vUAE]
conference_year                                                                 2023
event_currency_id                                                                  5
currency_id                                                                        2
event_date                                                                2023-06-26
impressions                                                                    24817
parent_conference                                                          Collision
previous_year_event_id                          72edb4ba-5f10-4e5f-b639-2bddda108dc2
next_year_event_id                              ad7088bb-6473-4f4

In [ ]:
marketing_spend_raw['reach_country_id'].value_counts()

reach_country_id
---    47816
CAN    10955
GBR    10579
USA    10397
DEU    10141
PRT     9907
NLD     9701
FRA     9668
IRL     9324
ESP     9238
BRA     9216
ITA     9115
POL     8975
BEL     8758
CHE     8438
SWE     7995
AUT     7711
MEX     7694
DNK     7677
ARE     7258
ARG     6849
QAT     6501
SAU     6466
CZE     6447
CHL     6386
PER     6357
LUX     6302
NOR     6287
GRC     6286
SRB     6151
HUN     6025
HRV     5958
FIN     5833
SVK     5654
OMN     5585
COL     5435
ROU     5237
SVN     5194
BGR     5101
IND     5084
PAK     5043
EGY     5018
PHL     5008
SGP     5006
JPN     4928
BIH     4903
LTU     4856
NGA     4854
MLT     4814
AUS     4785
MKD     4780
IDN     4756
ZAF     4719
TUR     4711
MAR     4693
URY     4686
KOR     4675
HKG     4618
TUN     4606
MYS     4571
THA     4532
EST     4525
KWT     4525
UKR     4501
ECU     4488
VNM     4452
GEO     4442
CYP     4419
LVA     4394
DZA     4377
GHA     4373
KEN     4333
BGD     4274
ALB     4243
LBN     4223
ISR     

In [ ]:
len(marketing_spend_raw['reach_country_id'])b

866071

In [ ]:
marketing_spend_raw.columns

Index(['Unnamed: 0', 'marketing_campaign_name', 'event_id',
       'event_salesforce_conference_ids', 'conference_year',
       'event_currency_id', 'currency_id', 'event_date', 'impressions',
       'parent_conference', 'previous_year_event_id', 'next_year_event_id',
       'classification_sub_group', 'event_tito_id', 'classification_id',
       'marketing_platform_id', 'clicks', 'date_id', 'reach',
       'event_salesforce_campaign_ids', 'event_avenger_id',
       'campaign_spend_eur', 'days_to_event', 'marketing_campaign_objective',
       'campaign_spend', 'marketing_ad_account_id', 'marketing_platform_name',
       'event_avenger_slug', 'last_ingestion_timestamp', 'weeks_to_event',
       'event_name', 'classification_group', 'marketing_campaign_id',
       'event_tito_slug', 'reach_country_id', 'marketing_ad_account_name'],
      dtype='object')

In [ ]:
marketing_spend_raw.head()

,Unnamed: 0,marketing_campaign_name,event_id,event_salesforce_conference_ids,conference_year,event_currency_id,currency_id,event_date,impressions,parent_conference,previous_year_event_id,next_year_event_id,classification_sub_group,event_tito_id,classification_id,marketing_platform_id,clicks,date_id,reach,event_salesforce_campaign_ids,event_avenger_id,campaign_spend_eur,days_to_event,marketing_campaign_objective,campaign_spend,marketing_ad_account_id,marketing_platform_name,event_avenger_slug,last_ingestion_timestamp,weeks_to_event,event_name,classification_group,marketing_campaign_id,event_tito_slug,reach_country_id,marketing_ad_account_name
0,0,Conf=Collision2023&Activity=open_source&Object...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,24817,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,Open Source,unknown,generalattendee-subsid_opensource,8,70.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,147.373190,54.0,CLICKS,161.565228,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,General Attendee- Subsid,1684929535866970982,unknown,---,CollisionConf
1,1,Conf=Collision2023&Activity=alpha&Objective=Le...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,6191,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,ALPHA,unknown,startups_alpha,8,28.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,55.369416,54.0,CLICKS,60.701491,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,Startups,1689310747829239719,unknown,---,CollisionConf
2,2,Conf=WebSummit2022&Activity=general_attendee&O...,f0009977-13e9-460a-9d34-3efca62a3b36,[a0J7R00000e9KOVUA2],2022,1,2,2022-11-01,613019,Web Summit,aebb4e42-a61d-4b12-8850-aa1d5bafb053,494e0d47-2838-43c4-bf1a-12aea8d65194,General Attendee,unknown,generalattendee_generalattendee,8,1516.0,2022-09-10,0,[7012X000001TH0oQAG],f0009977-13e9-460a-9d34-3efca62a3b36,1828.317769,52.0,CLICKS,1852.268732,t2_48ksux2y,Reddit,ws22,2024-10-07T03:50:02.858,7.0,Web Summit 2022,General Attendee,1523191475961491496,unknown,---,WebSummitConf
3,3,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,39309,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,127.0,2022-04-29,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,86.894148,52.0,CLICKS,95.809488,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf
4,4,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,102750,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,331.0,2022-04-28,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,238.412503,53.0,CLICKS,262.873626,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf


## This is the original attempt at the fitting

In [ ]:
combined_google.head().columns

Index(['facebook_impressions', 'facebook_reach_country_id',
       'facebook_campaign_spend_eur', 'linkedin_impressions',
       'linkedin_reach_country_id', 'linkedin_campaign_spend_eur',
       'google_impressions', 'google_reach_country_id',
       'google_campaign_spend_eur', 'bing_impressions',
       'bing_reach_country_id', 'bing_campaign_spend_eur',
       'ticktock_impressions', 'ticktock_reach_country_id',
       'ticktock_campaign_spend_eur', 'twitter_impressions',
       'twitter_reach_country_id', 'twitter_campaign_spend_eur',
       'instagram_impressions', 'instagram_reach_country_id',
       'instagram_campaign_spend_eur', 'reddit_impressions',
       'reddit_reach_country_id', 'reddit_campaign_spend_eur'],
      dtype='object')

In [ ]:
combined_google.to_csv('./ws_attempt/data/raw_data/combined_google.csv')

/home/ubuntu/.local/lib/python3.10/site-packages/meridian/data/load.py:1384: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


ValueError: Values of the `coord_to_columns` object ['bing_campaign_spend_eur', 'bing_impressions', 'clicks', 'date_id', 'facebook_campaign_spend_eur', 'facebook_impressions', 'facebook_reach_country_id', 'google_campaign_spend_eur', 'google_impressions', 'instagram_campaign_spend_eur', 'instagram_impressions', 'linkedin_campaign_spend_eur', 'linkedin_impressions', 'population', 'reddit_campaign_spend_eur', 'ticktock_campaign_spend_eur', 'ticktock_impressions', 'twitter_campaign_spend_eur', 'twitter_impressionsreddit_impressions'] should map to the DataFrame column names ['Unnamed: 0', 'campaign_spend', 'campaign_spend_eur', 'classification_group', 'classification_id', 'classification_sub_group', 'clicks', 'conference_year', 'currency_id', 'date_id', 'days_to_event', 'event_avenger_id', 'event_avenger_slug', 'event_currency_id', 'event_date', 'event_id', 'event_name', 'event_salesforce_campaign_ids', 'event_salesforce_conference_ids', 'event_tito_id', 'event_tito_slug', 'facebook_reach

In [ ]:
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    min_date = min(marketing_spend['date_id'])
    max_date = max(marketing_spend['date_id'])
    overall_date_range = pd.date_range(min_date, max_date)
    overall_date_range
    marketing_spend.index
    marketing_spend['date_id'] = pd.to_datetime(marketing_spend['date_id'])
    facebook = marketing_spend[marketing_spend['marketing_platform_name']=='FacebookAds'].add_prefix('facebook_')
    linkedin = marketing_spend[marketing_spend['marketing_platform_name']=='LinkedInAds'].add_prefix('linkedin_')
    google = marketing_spend[marketing_spend['marketing_platform_name']=='GoogleAds'].add_prefix('google_')
    bing = marketing_spend[marketing_spend['marketing_platform_name']=='BingAds'].add_prefix('bing_')
    ticktock = marketing_spend[marketing_spend['marketing_platform_name']=='TikTok'].add_prefix('ticktock_')
    twitter = marketing_spend[marketing_spend['marketing_platform_name']=='TwitterAds'].add_prefix('twitter_')    
    instagram = marketing_spend[marketing_spend['marketing_platform_name']=='Instagram'].add_prefix('instagram_')
    reddit = marketing_spend[marketing_spend['marketing_platform_name']=='Reddit'].add_prefix('reddit_')


    facebook['facebook_date_id'] = pd.to_datetime(facebook['facebook_date_id'])
    facebook = facebook.set_index(facebook['facebook_date_id'])
    facebook = facebook[['facebook_impressions','facebook_reach_country_id','facebook_campaign_spend_eur', 'facebook_date_id']]
    #facebook_clean = facebook.groupby(facebook.index).sum()

    linkedin = linkedin.set_index(linkedin['linkedin_date_id'])
    linkedin = linkedin.set_index(linkedin['linkedin_date_id'])
    linkedin = linkedin[['linkedin_impressions','linkedin_reach_country_id','linkedin_campaign_spend_eur', 'linkedin_date_id']]
    #linkedin_clean = linkedin.groupby(linkedin.index).sum()

    google = google.set_index(google['google_date_id'])
    google = google[['google_impressions','google_reach_country_id','google_campaign_spend_eur', 'google_date_id']]
    #google_clean = google.groupby(google.index).sum()

    bing = bing.set_index(bing['bing_date_id'])
    bing = bing[['bing_impressions','bing_reach_country_id','bing_campaign_spend_eur', 'bing_date_id']]
    #bing_clean = bing.groupby(bing.index).sum()

    ticktock = ticktock.set_index(ticktock['ticktock_date_id'])
    ticktock = ticktock[['ticktock_impressions','ticktock_reach_country_id','ticktock_campaign_spend_eur', 'ticktock_date_id']]
    #ticktock_clean = ticktock.groupby(ticktock.index).sum()

    twitter = twitter.set_index(twitter['twitter_date_id'])
    twitter = twitter[['twitter_impressions','twitter_reach_country_id','twitter_campaign_spend_eur', 'twitter_date_id']]
    #twitter_clean = twitter.groupby(twitter.index).sum()

    instagram = instagram.set_index(instagram['instagram_date_id'])
    instagram = instagram[['instagram_impressions','instagram_reach_country_id','instagram_campaign_spend_eur','instagram_date_id']]
    #instagram_clean = instagram.groupby(instagram.index).sum()

    reddit = reddit.set_index(reddit['reddit_date_id'])
    reddit = reddit[['reddit_impressions','reddit_reach_country_id','reddit_campaign_spend_eur', 'reddit_date_id']]
    #reddit_clean = reddit.groupby(reddit.index).sum()

    #combined_google_2 = pd.concat([facebook_clean, linkedin_clean, google_clean, bing_clean,
    #                 ticktock_clean, twitter_clean, instagram_clean, reddit_clean],
    #                axis=1)

   #combined_google = pd.concat([facebook, linkedin, google, bing,
    #                 ticktock, twitter, instagram, reddit],
                    axis=1)

    #return [google, facebook, twitter, ticktock, instagram, reddit, bing, linkedin, ticket_sales]

NameError: name 'marketing_spend' is not defined

In [ ]:
reddit

NameError: name 'reddit' is not defined

In [ ]:
facebook[facebook['facebook_date_id']=='2024-02-10']

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,facebook_date_id
facebook_date_id,,,,
2024-02-10,53,JAM,0.079983,2024-02-10
2024-02-10,134,MYS,0.979796,2024-02-10
2024-02-10,271,SOM,0.209956,2024-02-10
2024-02-10,187,OMN,1.249740,2024-02-10
2024-02-10,341,COL,1.479692,2024-02-10
2024-02-10,222,LKA,0.429911,2024-02-10
2024-02-10,2,LUX,0.000000,2024-02-10
2024-02-10,6,SUR,0.019996,2024-02-10
2024-02-10,1536,BGD,1.799626,2024-02-10


In [ ]:
merged_df = facebook.join(linkedin, how='outer')
merged_df = merged_df.join(twitter, how='outer')
merged_df = merged_df.join(google, how='outer')
merged_df = merged_df.join(bing, how='outer')
merged_df = merged_df.join(instagram, how='outer')
merged_df = merged_df.join(ticktock, how='outer')
merged_df = merged_df.join(reddit, how='outer')



: 

: 

: 

In [ ]:
merged_df.head()

,facebook_impressions,facebook_reach_country_id,facebook_campaign_spend_eur,facebook_date_id,linkedin_impressions,linkedin_reach_country_id,linkedin_campaign_spend_eur,linkedin_date_id
facebook_date_id,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaT,0.0,---,0.0,2022-01-03
2022-01-04,NaN,NaN,NaN,NaT,0.0,---,0.0,2022-01-04
2022-01-05,NaN,NaN,NaN,NaT,0.0,---,0.0,2022-01-05
2022-01-07,NaN,NaN,NaN,NaT,0.0,---,0.0,2022-01-07
2022-01-07,NaN,NaN,NaN,NaT,0.0,---,0.0,2022-01-07


# Try this again - from scratch!


In [ ]:
marketing_spend= pd.read_csv('/home/ubuntu/work/marketing/marketing_analytics/functionalised/data/raw_data/advertising_raw.csv')

/tmp/ipykernel_215184/1607298651.py:1: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend= pd.read_csv('/home/ubuntu/work/marketing/marketing_analytics/functionalised/data/raw_data/advertising_raw.csv')


In [ ]:
marketing_spend.head()

,Unnamed: 0,marketing_campaign_name,event_id,event_salesforce_conference_ids,conference_year,event_currency_id,currency_id,event_date,impressions,parent_conference,previous_year_event_id,next_year_event_id,classification_sub_group,event_tito_id,classification_id,marketing_platform_id,clicks,date_id,reach,event_salesforce_campaign_ids,event_avenger_id,campaign_spend_eur,days_to_event,marketing_campaign_objective,campaign_spend,marketing_ad_account_id,marketing_platform_name,event_avenger_slug,last_ingestion_timestamp,weeks_to_event,event_name,classification_group,marketing_campaign_id,event_tito_slug,reach_country_id,marketing_ad_account_name
0,0,Conf=Collision2023&Activity=open_source&Object...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,24817,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,Open Source,unknown,generalattendee-subsid_opensource,8,70.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,147.373190,54.0,CLICKS,161.565228,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,General Attendee- Subsid,1684929535866970982,unknown,---,CollisionConf
1,1,Conf=Collision2023&Activity=alpha&Objective=Le...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,6191,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,ALPHA,unknown,startups_alpha,8,28.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,55.369416,54.0,CLICKS,60.701491,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,Startups,1689310747829239719,unknown,---,CollisionConf
2,2,Conf=WebSummit2022&Activity=general_attendee&O...,f0009977-13e9-460a-9d34-3efca62a3b36,[a0J7R00000e9KOVUA2],2022,1,2,2022-11-01,613019,Web Summit,aebb4e42-a61d-4b12-8850-aa1d5bafb053,494e0d47-2838-43c4-bf1a-12aea8d65194,General Attendee,unknown,generalattendee_generalattendee,8,1516.0,2022-09-10,0,[7012X000001TH0oQAG],f0009977-13e9-460a-9d34-3efca62a3b36,1828.317769,52.0,CLICKS,1852.268732,t2_48ksux2y,Reddit,ws22,2024-10-07T03:50:02.858,7.0,Web Summit 2022,General Attendee,1523191475961491496,unknown,---,WebSummitConf
3,3,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,39309,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,127.0,2022-04-29,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,86.894148,52.0,CLICKS,95.809488,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf
4,4,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,102750,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,331.0,2022-04-28,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,238.412503,53.0,CLICKS,262.873626,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf


In [ ]:
def process_marketing_spend(df):
    """
    Process marketing spend dataframe to create platform-specific spend columns
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with marketing_platform_name and marketing_spend columns
    
    Returns:
    pandas.DataFrame: Processed dataframe with platform-specific spend columns
    """
    # List of platforms
    platforms = ['reddit', 'linkedin', 'facebook', 'google', 'bing', 
                'tiktok', 'twitter', 'instagram']
    
    # Create new spend columns initialized to 0
    for platform in platforms:
        df[f'{platform}_spend'] = 0
        
    # Iterate through each row and assign spend to appropriate column
    for idx, row in df.iterrows():
        platform = row['marketing_platform_name'].lower()
        if platform in platforms:
            df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
    
    return df

def process_marketing_impressions(df):
    """
    Process marketing spend dataframe to create platform-specific spend columns
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with marketing_platform_name and marketing_spend columns
    
    Returns:
    pandas.DataFrame: Processed dataframe with platform-specific spend columns
    """
    # List of platforms
    platforms = ['reddit', 'linkedin', 'facebook', 'google', 'bing', 
                'tiktok', 'twitter', 'instagram']
    
    # Create new spend columns initialized to 0
    for platform in platforms:
        df[f'{platform}_impressions'] = 0
        
    # Iterate through each row and assign spend to appropriate column
    for idx, row in df.iterrows():
        platform = row['marketing_platform_name'].lower()
        if platform in platforms:
            df.at[idx, f'{platform}_spend'] = row['impressions']
    
    return df

# Example usage:
# marketing_spend_processed = process_marketing_spend(marketing_spend)

# To verify the results:
# print(marketing_spend_processed.head())


In [ ]:
test = process_marketing_spend(marketing_spend)
test = process_marketing_impressions(test)

/tmp/ipykernel_215184/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '147.37319' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
/tmp/ipykernel_215184/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2231.85' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
/tmp/ipykernel_215184/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']


In [ ]:
test.head()

,Unnamed: 0,marketing_campaign_name,event_id,event_salesforce_conference_ids,conference_year,event_currency_id,currency_id,event_date,impressions,parent_conference,previous_year_event_id,next_year_event_id,classification_sub_group,event_tito_id,classification_id,marketing_platform_id,clicks,date_id,reach,event_salesforce_campaign_ids,event_avenger_id,campaign_spend_eur,days_to_event,marketing_campaign_objective,campaign_spend,marketing_ad_account_id,marketing_platform_name,event_avenger_slug,last_ingestion_timestamp,weeks_to_event,event_name,classification_group,marketing_campaign_id,event_tito_slug,reach_country_id,marketing_ad_account_name,reddit_spend,linkedin_spend,facebook_spend,google_spend,bing_spend,tiktok_spend,twitter_spend,instagram_spend,reddit_impressions,linkedin_impressions,facebook_impressions,google_impressions,bing_impressions,tiktok_impressions,twitter_impressions,instagram_impressions
0,0,Conf=Collision2023&Activity=open_source&Object...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,24817,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,Open Source,unknown,generalattendee-subsid_opensource,8,70.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,147.373190,54.0,CLICKS,161.565228,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,General Attendee- Subsid,1684929535866970982,unknown,---,CollisionConf,24817.0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0
1,1,Conf=Collision2023&Activity=alpha&Objective=Le...,20ed6ee7-0a8f-41b7-8497-a5733dc61431,[a0J7R00000e9R6vUAE],2023,5,2,2023-06-26,6191,Collision,72edb4ba-5f10-4e5f-b639-2bddda108dc2,ad7088bb-6473-4f4c-8ccf-567dbb9c306b,ALPHA,unknown,startups_alpha,8,28.0,2023-05-03,0,[7017R000001yUJ0QAM],20ed6ee7-0a8f-41b7-8497-a5733dc61431,55.369416,54.0,CLICKS,60.701491,t2_2y0f1uhq,Reddit,cc23,2024-10-07T03:50:02.858,7.0,2023 Collision,Startups,1689310747829239719,unknown,---,CollisionConf,6191.0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0
2,2,Conf=WebSummit2022&Activity=general_attendee&O...,f0009977-13e9-460a-9d34-3efca62a3b36,[a0J7R00000e9KOVUA2],2022,1,2,2022-11-01,613019,Web Summit,aebb4e42-a61d-4b12-8850-aa1d5bafb053,494e0d47-2838-43c4-bf1a-12aea8d65194,General Attendee,unknown,generalattendee_generalattendee,8,1516.0,2022-09-10,0,[7012X000001TH0oQAG],f0009977-13e9-460a-9d34-3efca62a3b36,1828.317769,52.0,CLICKS,1852.268732,t2_48ksux2y,Reddit,ws22,2024-10-07T03:50:02.858,7.0,Web Summit 2022,General Attendee,1523191475961491496,unknown,---,WebSummitConf,613019.0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0
3,3,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,39309,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,127.0,2022-04-29,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,86.894148,52.0,CLICKS,95.809488,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf,39309.0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0
4,4,Conf=Collision2022&Activity=general_attendee&O...,72edb4ba-5f10-4e5f-b639-2bddda108dc2,[a0J2X00000dYpCfUAK],2022,5,2,2022-06-20,102750,Collision,2021-collision,20ed6ee7-0a8f-41b7-8497-a5733dc61431,General Attendee,unknown,generalattendee_generalattendee,8,331.0,2022-04-28,0,[7012X000001TULcQAO],72edb4ba-5f10-4e5f-b639-2bddda108dc2,238.412503,53.0,CLICKS,262.873626,t2_2y0f1uhq,Reddit,cc22,2024-10-07T03:50:02.858,7.0,Collision 2022,General Attendee,1426082416056841806,unknown,---,CollisionConf,102750.0,0,0,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0


In [ ]:
test.to_csv('./ws_attempt/data/raw_data/marketing_spend_processed_new_approach.csv')

In [ ]:
test.columns

Index(['Unnamed: 0', 'marketing_campaign_name', 'event_id',
       'event_salesforce_conference_ids', 'conference_year',
       'event_currency_id', 'currency_id', 'event_date', 'impressions',
       'parent_conference', 'previous_year_event_id', 'next_year_event_id',
       'classification_sub_group', 'event_tito_id', 'classification_id',
       'marketing_platform_id', 'clicks', 'date_id', 'reach',
       'event_salesforce_campaign_ids', 'event_avenger_id',
       'campaign_spend_eur', 'days_to_event', 'marketing_campaign_objective',
       'campaign_spend', 'marketing_ad_account_id', 'marketing_platform_name',
       'event_avenger_slug', 'last_ingestion_timestamp', 'weeks_to_event',
       'event_name', 'classification_group', 'marketing_campaign_id',
       'event_tito_slug', 'reach_country_id', 'marketing_ad_account_name',
       'reddit_spend', 'linkedin_spend', 'facebook_spend', 'google_spend',
       'bing_spend', 'tiktok_spend', 'twitter_spend', 'instagram_spend',
       'red

In [ ]:


coord_to_columns = load.CoordToColumns(
    time='date_id',
    geo='reach_country_id',
    controls=[],
    #population='population',
    kpi='clicks',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'linkedin_impressions',
        'google_impressions',
        'bing_impressions',
        'tiktok_impressions',
        'twitter_impressions',
        'reddit_impressions',
        'instagram_impressions',
    ],
    media_spend=[
        'facebook_spend',
        'linkedin_spend',
        'google_spend',
        'bing_spend',
        'tiktok_spend',
        'twitter_spend',
        'reddit_spend',
        'instagram_spend',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'linkedin_impressions': 'LinkedIn',
    'google_impressions': 'Google',
    'bing_impressions': 'Bing',
    'tiktok_impressions': 'TickTock',
    'twitter_impressions': 'Twitter',
    'reddit_impressions': 'Reddit',
    'instagram_impressions': 'Instagram',
}
correct_media_spend_to_channel = {
    'facebook_spend': 'Facebook',
    'linkedin_spend': 'LinkedIn',
    'google_spend': 'Google',
    'bing_spend': 'Bing',
    'tiktok_spend': 'TickTock',
    'twitter_spend': 'Twitter',
    'reddit_spend': 'Reddit',
    'instagram_spend': 'Instagram',
}


#loader = load.CsvDataLoader(
#    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
#    kpi_type='non_revenue',
#    coord_to_columns=coord_to_columns,
#    media_to_channel=correct_media_to_channel,
#    media_spend_to_channel=correct_media_spend_to_channel,
#)

loader = load.CsvDataLoader(
    csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


data = loader.load()

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000, seed=1)

/home/ubuntu/.local/lib/python3.10/site-packages/meridian/data/load.py:1384: DtypeWarning: Columns (26,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


ValueError: Duplicate entries found in the 'time' column.

In [ ]:
# Read the CSV file
import pandas as pd
df = pd.read_csv("/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach.csv")

# Check for duplicates in the combination of date and geo
duplicates = df.groupby(['date_id', 'reach_country_id']).size().reset_index(name='count')
duplicates = duplicates[duplicates['count'] > 1]

if len(duplicates) > 0:
    print("Found duplicate date-geo combinations:")
    print(duplicates)
    
    # If you need to aggregate duplicate date-geo combinations
    df_aggregated = df.groupby(['date_id', 'reach_country_id']).agg({
        'clicks': 'sum',
        'facebook_impressions': 'sum',
        'linkedin_impressions': 'sum',
        'google_impressions': 'sum',
        'bing_impressions': 'sum',
        'tiktok_impressions': 'sum',
        'twitter_impressions': 'sum',
        'reddit_impressions': 'sum',
        'instagram_impressions': 'sum',
        'facebook_spend': 'sum',
        'linkedin_spend': 'sum',
        'google_spend': 'sum',
        'bing_spend': 'sum',
        'tiktok_spend': 'sum',
        'twitter_spend': 'sum',
        'reddit_spend': 'sum',
        'instagram_spend': 'sum'
    }).reset_index()
    
    df_aggregated.to_csv("/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach_aggregated.csv", 
                         index=False)
    
    # Use the aggregated file
    loader = load.CsvDataLoader(
        csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach_aggregated.csv",
        kpi_type='non_revenue',
        coord_to_columns=coord_to_columns,
        media_to_channel=correct_media_to_channel,
        media_spend_to_channel=correct_media_spend_to_channel,
    )
else:
    print("No duplicate date-geo combinations found")
    # Use the original file
    loader = load.CsvDataLoader(
        csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach.csv",
        kpi_type='non_revenue',
        coord_to_columns=coord_to_columns,
        media_to_channel=correct_media_to_channel,
        media_spend_to_channel=correct_media_spend_to_channel,
    )


/tmp/ipykernel_215184/1827918629.py:3: DtypeWarning: Columns (26,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach.csv")


Found duplicate date-geo combinations:
           date_id reach_country_id  count
1       2022-01-02              ARE      2
2       2022-01-02              ARG      2
4       2022-01-02              AUT      2
6       2022-01-02              BEL      2
13      2022-01-02              CAN      2
...            ...              ...    ...
195544  2024-10-07              NLD      2
195545  2024-10-07              POL      2
195546  2024-10-07              PRT      3
195547  2024-10-07              QAT      2
195550  2024-10-07              USA      2

[157314 rows x 3 columns]


ValueError: Values in the 'time' column not consistent across different geos.

In [ ]:
# Get all unique dates and geos
all_dates = df['date_id'].unique()
all_geos = df['reach_country_id'].unique()

# Create a complete date-geo combination DataFrame
index = pd.MultiIndex.from_product([all_dates, all_geos], 
                                 names=['date_id', 'reach_country_id'])
df_complete = df.set_index(['date_id', 'reach_country_id']).reindex(index)

# Fill missing values with 0
df_complete = df_complete.fillna({
    'clicks': 0,
    'facebook_impressions': 0,
    'linkedin_impressions': 0,
    'google_impressions': 0,
    'bing_impressions': 0,
    'tiktok_impressions': 0,
    'twitter_impressions': 0,
    'reddit_impressions': 0,
    'instagram_impressions': 0,
    'facebook_spend': 0,
    'linkedin_spend': 0,
    'google_spend': 0,
    'bing_spend': 0,
    'tiktok_spend': 0,
    'twitter_spend': 0,
    'reddit_spend': 0,
    'instagram_spend': 0
}).reset_index()

# Save the complete dataset
df_complete.to_csv("/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_complete.csv", 
                  index=False)

# Use the complete dataset
loader = load.CsvDataLoader(
    csv_path="/home/ubuntu/work/marketing/google/ws_attempt/data/raw_data/marketing_spend_processed_complete.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


ValueError: cannot handle a non-unique multi-index!